In [1]:
from scihubdownloader import SciHub
from fileprocessor import FileProcessor

In [2]:
#import os
#os.mkdir("SciHub")
#os.mkdir("SciHub/SERB")

In [3]:
download_path = '/opt/lampp/htdocs/registration/SciHub/SERB'
bib_path = "/home/dibya/Downloads/serb4.bib"
downloader = SciHub(bibtex_file=bib_path, download_folder=download_path, json_file="vaefile.json")

In [4]:
downloader.download()

  0%|          | 0/41 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [103]:
fp = FileProcessor(bibfile_path=bib_path, file_data_path="vaefile.json")
fp.extract_data(check_path="SciHub/SERB/", rel_path = "SciHub/SERB/")

In [104]:
fp.write_to_file("index.php")

In [9]:
from scihub import SciHub

sh = SciHub()

# exactly the same thing as fetch except downloads the articles to disk
# if no path given, a unique name will be used as the file name


INFO:Sci-Hub:Failed to fetch pdf with identifier https://doi.org/10.1016/j.csbj.2021.08.011 (resolved url None) due to request exception.


In [15]:
result = sh.fetch('https://www.sciencedirect.com/science/article/pii/S2001037021003421')

INFO:Sci-Hub:Failed to fetch pdf with identifier https://www.sciencedirect.com/science/article/pii/S2001037021003421 (resolved url None) due to request exception.


In [17]:
import random
import string

def generate_name(length):
    letters = string.ascii_lowercase
    name = ''.join(random.choice(letters) for _ in range(length))
    return name.capitalize()

# Example usage
for i in range(5):
    print(generate_name(10))


Zfwbovrmhh
Hrrkkiftts
Cytbwgqfwa
Fqezbvddie
Zdhjooktyg


In [100]:
import bibtexparser
import contextlib
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import undetected_chromedriver as uc
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import subprocess as sp
import numpy as np
import json
import os
import time
from tqdm import tqdm
from tqdm.auto import trange
from joblib import Parallel, delayed

class SciHub:
    def __init__(self,  bibtex_file, json_file, download_folder):
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        options.add_argument("--no-sandbox")
        options.add_argument('window-size=1920x1080')
        self.driver = uc.Chrome(exec_file = "/home/dibya/Dibyendu/Test/HTML/selenium/chromedriver/linux64/109.0.5414.74/chromedriver",options = options)
        with open(bibtex_file) as bibfile:
            self.bib_db = bibtexparser.load(bibfile)
        self.json_file = json_file
        self.download_folder = download_folder

    def downloadLink(self, url):
        self.driver.get(url)
        try:
            button = WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//button[text()='↓ save']")))
        except Exception:
            time.sleep(10)
            self.driver.get(url)
            button = WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//button[text()='↓ save']")))        

        down_str = button.get_attribute('onclick')
        down_link = down_str.split('href=')[1].replace("'", "")
        down_link = down_link.replace('?download=true', "")
        if down_link.startswith("//"):
            down_link = down_link[2:]
        elif down_link.startswith("/"):
            down_link = f'https://sci-hub.ru{down_link}'
        else:
            down_link = down_link
        return down_link
    

    def generate_name(self, length):
        letters = string.ascii_lowercase
        name = ''.join(random.choice(letters) for _ in range(length))
        return f"{name.capitalize()}.pdf"

    def fileDownloader(self, doi):
        #cmd = f'cd {self.download_folder} && wget {down_link}'
        #result = sp.run([cmd], shell=True, stderr=sp.DEVNULL, stdout=sp.DEVNULL)
        file_name = self.generate_name(10)

        while True:
            if os.path.isfile(f"{self.download_folder}/{file_name}"):
                file_name = self.generate_name(10)
            else:
                break
        #print(file_name)

        cmd = f"scidownl download --doi {doi} --out {self.download_folder}/{file_name}"
        r = sp.run([cmd], shell=True, stderr=sp.DEVNULL, stdout=sp.DEVNULL)
        
        return file_name if r.returncode == 0 else None
        
    def single_fetcher(self, i):
        with contextlib.suppress(Exception):
            url = "https://sci-hub.ru/" + self.bib_db.entries[i]['doi']
            with contextlib.suppress(Exception):
                dlink = self.downloadLink(url)
                #allLinks.append(dlink)
                flink = dlink.split("/")[-1]
                #file_json[i] = flink
                time.sleep(np.random.randint(10))
                return dlink, flink

    
    def download(self):
        file_json = {}
        allLinks = []
        
        import contextlib
        doi_list = []
        index_list = []
        for i in range(len(downloader.bib_db.entries)):
            with contextlib.suppress(Exception):
                doi = downloader.bib_db.entries[i]['doi']
                doi_list.append(doi)
                index_list.append(i)
        results = Parallel(n_jobs=-1,  prefer="threads")(
            delayed(self.fileDownloader)(doi_list[i])
            for i in trange(len(doi_list))
        )
        
        res = {index_list[i]: results[i] for i in range(len(results)) if results[i] is not None}
        
        with open(self.json_file, "w") as f:
            f.write(json.dumps(res))
        return res


In [101]:
download_path = '/opt/lampp/htdocs/registration/SciHub/SERB'
bib_path = "/home/dibya/Downloads/SERB.bib"
downloader = SciHub(bibtex_file=bib_path, download_folder=download_path, json_file="vaefile.json")

In [102]:
i, r = downloader.download()

  0%|          | 0/144 [00:00<?, ?it/s]

ValueError: too many values to unpack (expected 2)

In [77]:
np.where(r == "None")

(array([], dtype=int64),)

In [97]:
json.dum

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 35,
 36,
 37,
 38,
 39,
 40]

In [96]:
res

{0: 'Jelznirwtn.pdf',
 1: 'Ankcnndbpb.pdf',
 2: 'Tqefkblmsz.pdf',
 3: 'Oxcubpuhsp.pdf',
 4: 'Ghyzehaaei.pdf',
 5: 'Oiwenzzyse.pdf',
 6: 'Jfroaaozqb.pdf',
 7: 'Pfvdrkeimx.pdf',
 8: 'Uklyyxteee.pdf',
 9: 'Udpcepgxex.pdf',
 10: 'Eduiurrecx.pdf',
 11: 'Pfvkobuhpu.pdf',
 12: 'Lhgmlazqyx.pdf',
 13: 'Ajpxtdgezq.pdf',
 14: 'Qepsqooyjf.pdf',
 17: 'Ixrhqkhmld.pdf',
 18: 'Ilievfyebg.pdf',
 19: 'Rplgfznezc.pdf',
 22: 'Alepgemkbm.pdf',
 23: 'Qjfvbsoqzd.pdf',
 24: 'Kkzjmqmvrg.pdf',
 25: 'Ungkfslikn.pdf',
 26: 'Brqqqxxvzw.pdf',
 27: 'Fesipawxaa.pdf',
 28: 'Bnwgdqckyp.pdf',
 29: 'Xnhatahumf.pdf',
 30: 'Sxajjoalnx.pdf',
 31: 'Fxhljtwvwl.pdf',
 32: 'Jyhgqjkqif.pdf',
 33: 'Ukvifxxuxh.pdf',
 34: 'Ioaqjbisef.pdf',
 35: 'Vtjannaati.pdf',
 37: 'Dehvxhdjrc.pdf',
 39: 'Rbdlqseift.pdf'}

In [87]:
downloader.bib_db.entries[38]

{'doi': '10.1007/978-981-15-3514-7_36',
 'author': 'S. Suma and P. Nataraja and M. Sharma',
 'abstract': 'S2 TL;DR: The paper aims to pursue the possibilities of predicting Internet addiction based on a set of predictor variables such that there exists a strong relationship between the parameters considered to have an influence toward problematic Internet usage.',
 'url': 'https://www.semanticscholar.org/paper/6db8df523e00fdf34afd332191615bca42fd38ac',
 'year': '2020',
 'title': 'Internet Addiction Predictor: Applying Machine Learning in Psychology',
 'ENTRYTYPE': 'article',
 'ID': '6db8df523e00fdf34afd332191615bca42fd38ac'}